<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch10/section_92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 91. 続きのテキスト予測
“The movie was full of”に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。
### temperature（温度）パラメータとは
- softmax関数に渡す定数（例としてTとしている）
$$
p_i = softmax(\frac{z_i}{T})
$$
### 温度パラメータの変更による影響
- 温度パラメータはGPTが応答を生成する際に、生成される応答の多様性を制御するために調整される。
- 温度パラメータが高いほど、よりランダムな応答が生成される。
- 逆に温度パラメータが低いほど、より確信度の高い応答が生成される。

補足
- 今回はPADとEOSを設定する必要はないが慣習として設定する

In [2]:
!pip install -U transformers

### GenerateConfigとは？
- transformersライブラリに用意されているテキスト生成専用の設定クラス
- model.generate()の引数に設定して、デコーディング時の挙動をまとめて指定するためのクラス


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

In [4]:
model_id = "gpt2"
device = "cuda" if torch.cuda.is_available() else "cpu"
temperatures = [1.0, 0.7, 0.5, 0.1]
max_new_tokens = 50

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

In [8]:
text = "The movie was full of"

# paddingとtruncationは慣習的に設定している
input_ids = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)

### step_logitsとは？
- iステップ目の全語彙に対するlogits（softmax前の生スコア）

### softmaxだけだと？
- 各トークンの確率 \(p_i\) を得られる
- ただし、確率は非常に小さくなりやすく、数値的に不安定になりやすい

### log_softmaxとは？
- softmax の結果を log にしたもの（= 各トークンの対数確率）
- 数式的には  
  $$
  \log p_i = \log(\frac{e^{z_i}}{\sum_j e^{z_j}})
           = z_i - \log(\sum_j e^{z_j})
  $$
- $$\log(\sum_j e^{z_j})$$ は全クラス共通の正規化項（log-sum-exp）であり、各トークンに同じ値を引くだけなので順位（argmax）は変わらない
- 実装では最大値を引くなどの工夫により、exp のオーバーフローや確率のアンダーフローを防ぎ、数値的に安定に計算できる（要するにlong型などの限界値に到達するのを防げる）
- 対数確率を用いることで、文章全体の尤度を「確率の積」ではなく「対数確率の和」として扱える

### scoresはどんな順番で入っているか？
- 生成された文章の順序そのまま
- scores[i] は「i番目に生成されたトークン」に対応する全語彙の logits
- ただし、プロンプトとして入力した文章は含まれない

### log_prodsについて
- log_prodsは語彙数分の確率が返される
- token_idでそれを指定することによって対象の確率を取得できる

In [10]:
print("Greedy method")
with torch.no_grad():
  generation_config = GenerationConfig(
      max_new_tokens=max_new_tokens,
      pad_token_id=tokenizer.pad_token_id,
      eos_token_id=tokenizer.eos_token_id,
      do_sample=False, # beamが指定されていない場合はGreedy
      return_dict_in_generate=True,
      output_scores=True
  )
  output = model.generate(**input_ids, generation_config=generation_config)
  sequences = output.sequences[0]
  scores = output.scores

  prompt_len = input_ids["input_ids"].shape[1]
  new_token_ids = sequences[prompt_len:] # 生成した部分だけ取得

  for i, token_id in enumerate(new_token_ids):
    step_logits = scores[i][0]
    log_prods = torch.log_softmax(step_logits, dim=-1)
    logp = log_prods[token_id].item()
    p = torch.exp(log_prods[token_id]).item() # 対数で取得した確率を元の確率に戻す

    token_str = tokenizer.decode([token_id]).replace("\n", "\\n")
    print(f"{i+1:02d} token={token_str!r:12s} p={p:.6f} logp={logp:.6f}")

Greedy method
01 token=' jokes'     p=0.021892 logp=-3.821651
02 token=' and'       p=0.289225 logp=-1.240552
03 token=' jokes'     p=0.098500 logp=-2.317703
04 token=' about'     p=0.205553 logp=-1.582050
05 token=' how'       p=0.099715 logp=-2.305439
06 token=' the'       p=0.084638 logp=-2.469367
07 token=' movie'     p=0.036412 logp=-3.312869
08 token=' was'       p=0.296338 logp=-1.216254
09 token=' a'         p=0.067677 logp=-2.693010
10 token=' joke'      p=0.173508 logp=-1.751529
11 token='.'          p=0.280387 logp=-1.271584
12 token=' It'        p=0.123003 logp=-2.095544
13 token=' was'       p=0.519723 logp=-0.654460
14 token=' a'         p=0.149313 logp=-1.901710
15 token=' joke'      p=0.268988 logp=-1.313089
16 token=' about'     p=0.424149 logp=-0.857670
17 token=' how'       p=0.174166 logp=-1.747744
18 token=' the'       p=0.123646 logp=-2.090337
19 token=' movie'     p=0.616074 logp=-0.484388
20 token=' was'       p=0.634955 logp=-0.454201
21 token=' a'         p=0.